## Generate Models

This code generates the models for a rectangular wing with a FWTD, it creates a series of models to test different non-linearaties

In [3]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import odeint,LSODA,BDF,solve_ivp
from scipy.linalg import eig
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RectWing.Models as rwm

me.mechanics_printing()



In [4]:
def Model_0(b_modes,t_modes):
    return rwm.GenRectWingModel(b_modes,t_modes,fwt_free=False,iwt=True,iwb=False,fwt_frot=False)
def Model_1(b_modes,t_modes):
    return rwm.GenRectWingModel(b_modes,t_modes,fwt_free=True,iwt=False,iwb=False,fwt_frot=False)
def Model_2(b_modes,t_modes):
    return rwm.GenRectWingModel(b_modes,t_modes,fwt_free=True,iwt=True,iwb=False,fwt_frot=False)
def Model_3(b_modes,t_modes):
    return rwm.GenRectWingModel(b_modes,t_modes,fwt_free=True,iwt=True,iwb=True,fwt_frot=False)
def Model_4(b_modes,t_modes):
    return rwm.GenRectWingModel(b_modes,t_modes,fwt_free=True,iwt=True,iwb=True,fwt_frot=True)
def Model_5(b_modes,t_modes):
    model,p = Model_4(b_modes,t_modes)
    return model.linearise(p).subs({i:0 for i in p.fp}),p
def Model_6(b_modes,t_modes):
    model,p = rwm.GenRectWingModel(b_modes,t_modes,fwt_free=True,iwt=True,iwb=False,fwt_frot=True)
    return model.linearise(p).subs({i:0 for i in p.fp}),p

ModelFactory = {
    0:Model_0,
    1:Model_1,
    2:Model_2,
    3:Model_3,
    4:Model_4,
    5:Model_5,
    6:Model_6
}


def MakeModel(k,v,b_modes,t_modes):
    model,p = v(b_modes,t_modes)
    model.to_file(f'{b_modes}B{t_modes}T-M{k}.py')
    return k

In [3]:
b_modes = 3
t_modes = 3
import multiprocessing as mp
from joblib import Parallel, delayed

pool = mp.Pool(mp.cpu_count())


for k,v in ModelFactory.items():
    pool.apply_async(MakeModel,args=(k,v,b_modes,t_modes))
pool.close()
pool.join()

In [5]:
%time MakeModel(4,Model_4,3,3)

CPU times: user 55 s, sys: 314 ms, total: 55.3 s
Wall time: 55.8 s
